<a href="https://colab.research.google.com/github/Aqui1983/Module_13_Activities/blob/main/Multiclass_in_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass Prediction in Deep Learning Models

## Read in the Data

In [1]:
# Import required packages
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [4]:
# Read in the data
df = pd.read_csv('/content/meet_or_beat.csv')
df.head()

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns
0,2.01,1.67,11.0,0.051444,0.018585
1,0.17,0.19,6.0,0.112955,-0.000510
2,-0.07,0.14,4.0,0.077167,-0.046104
3,0.48,0.51,8.0,-0.006130,-0.004899
4,-0.24,-0.27,9.0,0.089762,-0.025466


In [5]:
# Check dataset size
df.shape

(71968, 5)

In [ ]:
# Preview dataframe


## Preparing the Data

In [13]:
# Generate the categorical outcome variable
df['earnings_outcome'] = np.nan

df.loc[(df['EPS']==df['forecasted_eps']), 'earnings_outcome']='meet'
df.loc[(df['EPS']>df['forecasted_eps']), 'earnings_outcome']='beat'
df.loc[(df['EPS']<df['forecasted_eps']), 'earnings_outcome']='lose'

df

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns,earnings_outcome
0,2.01,1.67,11.0,0.051444,0.018585,beat
1,0.17,0.19,6.0,0.112955,-0.000510,lose
2,-0.07,0.14,4.0,0.077167,-0.046104,lose
3,0.48,0.51,8.0,-0.006130,-0.004899,lose
4,-0.24,-0.27,9.0,0.089762,-0.025466,beat
...,...,...,...,...,...,...
71963,0.31,0.30,4.0,0.006035,0.016854,beat
71964,-0.65,-0.66,3.0,0.179327,-0.039052,beat
71965,0.27,0.28,3.0,0.059002,0.141599,lose
71966,0.11,0.10,4.0,-0.035755,0.026346,beat


In [14]:
# Preview the output variable 
y = df['earnings_outcome']
y

0        beat
1        lose
2        lose
3        lose
4        beat
         ... 
71963    beat
71964    beat
71965    lose
71966    beat
71967    meet
Name: earnings_outcome, Length: 71968, dtype: object

In [15]:
# Encode earnings labels to integers
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder().fit(y)
encoded_y = encoder.transform(y)

encoded_y

array([0, 1, 1, ..., 1, 0, 2])

In [19]:
# Save the unique number of labels for future use
number_of_classes = len(list(y.drop_duplicates()))
number_of_classes

3

In [21]:
# Convert labeled integers to a Keras `categorical` data type
from keras.utils.np_utils import to_categorical

y_categorical = to_categorical(encoded_y, num_classes=number_of_classes)

In [23]:
# Specify X (predictor) variables
X = df[['forecasted_eps', 'before_total_returns','noOfEsts']]
X

,forecasted_eps,before_total_returns,noOfEsts
0,1.67,0.018585,11.0
1,0.19,-0.000510,6.0
2,0.14,-0.046104,4.0
3,0.51,-0.004899,8.0
4,-0.27,-0.025466,9.0
...,...,...,...
71963,0.30,0.016854,4.0
71964,-0.66,-0.039052,3.0
71965,0.28,0.141599,3.0
71966,0.10,0.026346,4.0


In [24]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state=1)

In [ ]:
# Preview the encoded data we're trying to predict


In [25]:
# Check for class balance
pd.DataFrame(y_train).sum()

0    32924.0
1    16878.0
2     4174.0
dtype: float32

In [27]:
# Save the count of unique predictor variables for use in model
number_predictor_variables = len(X.columns)
number_predictor_variables

3

## Building, Fitting and Predicting with Multiple Classes

In [28]:
# Build the neural network layers
model = Sequential()
model.add(Dense(units=9, input_dim=number_predictor_variables, activation='relu'))
model.add(Dense(units=6, activation='relu'))


In [29]:
# Add the final output layer
# With the # of possible outputs equal to the number classes 

model.add(Dense(number_of_classes, activation='softmax'))

In [30]:
# Compile the model (with multi-class specific parameters)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [31]:
# Summarise the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 36        
                                                                 
 dense_1 (Dense)             (None, 6)                 60        
                                                                 
 dense_2 (Dense)             (None, 3)                 21        
                                                                 
Total params: 117
Trainable params: 117
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Fit the model
model.fit(X_train, y_train, epochs=20, batch_size=1000, shuffle=True)

Epoch 1/20
54/54 [==============================] - 1s 2ms/step - loss: 1.0004 - categorical_accuracy: 0.5550
Epoch 2/20
54/54 [==============================] - 0s 2ms/step - loss: 0.9211 - categorical_accuracy: 0.6095
Epoch 3/20
54/54 [==============================] - 0s 3ms/step - loss: 0.8904 - categorical_accuracy: 0.6100
Epoch 4/20
54/54 [==============================] - 0s 2ms/step - loss: 0.8743 - categorical_accuracy: 0.6100
Epoch 5/20
54/54 [==============================] - 0s 3ms/step - loss: 0.8676 - categorical_accuracy: 0.6101
Epoch 6/20
54/54 [==============================] - 0s 3ms/step - loss: 0.8609 - categorical_accuracy: 0.6101
Epoch 7/20
54/54 [==============================] - 0s 3ms/step - loss: 0.8561 - categorical_accuracy: 0.6101
Epoch 8/20
54/54 [==============================] - 0s 3ms/step - loss: 0.8514 - categorical_accuracy: 0.6103
Epoch 9/20
54/54 [==============================] - 0s 3ms/step - loss: 0.8472 - categorical_accuracy: 0.6101
Epoch 10/2

In [33]:
# Evaluate model on the test data
model.evaluate(X_test, y_test)

563/563 [==============================] - 1s 2ms/step - loss: 0.8401 - categorical_accuracy: 0.6147


[0.8400772213935852, 0.6147176623344421]

In [34]:
# Save predictions on the test data
predictions = model.predict(X_test)
predictions

563/563 [==============================] - 1s 1ms/step


array([[0.59103215, 0.32935098, 0.07961687],
       [0.7857178 , 0.16296795, 0.05131424],
       [0.5893136 , 0.36262536, 0.04806098],
       ...,
       [0.6673323 , 0.2700907 , 0.06257684],
       [0.559611  , 0.33957112, 0.1008179 ],
       [0.50574636, 0.417509  , 0.07674465]], dtype=float32)

In [35]:
# Get the most likely prediction for each observation
most_likely = np.argmax(predictions, axis=1)
most_likely

array([0, 0, 0, ..., 0, 0, 0])

In [36]:
# Convert most likely category back to original labels
most_likely = encoder.inverse_transform((most_likely))
most_likely

array(['beat', 'beat', 'beat', ..., 'beat', 'beat', 'beat'], dtype=object)

In [37]:
# Evaluate prediction balance
pd.DataFrame(most_likely).value_counts()

beat    17980
lose       12
dtype: int64